# Práctico 3: Análisis descriptivo y comparativo sobre diversas redes

# Introducción

Hasta ahora, hemos trabajado con grafos "conocidos". Sin embargo, muchas veces nos encontramos con un grafo del que no tenemos información y nuestro objectivo es entenderlo mejor.

Esto es análogo al trabajo de un cientifico de datos enfrentandose por primera vez a datos tabulares. En este caso, tenemos que encontrar alternativas a los promedios y las varianzas.

En este práctico vamos a trabajar con 12 grafos "desconocidos" y vamos a intentar aprender de ellos e identificar, en la medida de lo posible, cuáles se parecen entre sí.

In [ ]:
!pip install python-igraph
!pip install cairocffi
!pip install powerlaw

In [ ]:
import igraph as ig
import matplotlib.pyplot as plt
import statistics
import pandas as pd
import numpy as np
import time
import powerlaw
from pathlib import Path

from collections import defaultdict

Lo primero que vamos a hacer es descargar los 12 grafos que vamos a usar

In [ ]:
#!rm *.graphml*

In [ ]:
!printf '%s\n' {a..m} | xargs -I{} wget -q "https://raw.githubusercontent.com/prbocca/na101_master/master/homework_03_descriptive/graph_files/{}.graphml" & echo "Listo!"

Una vez descargados los 12 grafos vamos a cargarlos!

Como podrán ver, los grafos tienen nombres de letras. Esto es para que el processo de identificación se haga solo en base a las propiedades de los grafos.

In [ ]:
paths = sorted(Path("/content").glob("*.graphml"))
graphs = {}
for pth in paths:
  name = pth.name.split(".")[0]
  g = ig.load(str(pth))
  graphs[name] = g 

In [ ]:
for k, g in graphs.items():
  print(k, g.summary())

El objectivo de este práctico va a ser el cálculo de distintas métricas descriptivas de interés sobre los grafos obtenidos.

# 1) Verificar si los grafos son conexos

In [ ]:
for key, g in graphs.items():
  print(key, g.is_connected())

# 2) Medidas de centralidad en un grafo

Para los 3 experimentos que siguen, vamos a usar algunas de las metricas de centralidad que aprendimos durante el práctico anterior.

Entre ellas:

* La distribución de grado
* El grado promedio de los vecinos de un nodo
* La distribución de betweenness de cada grafo

## 2.1) Distribución de grado

In [ ]:
fig, ax = plt.subplots(4, 3, figsize=(20, 20))

for i, (k, g) in enumerate(graphs.items()):
  degree = g.degree()
  ax_ = ax[i // 3, i % 3]
  ax_.hist(degree)
  ax_.set_title(f"Graph {k} -- |V|= {g.vcount()}, |E| = {g.ecount()}")

In [ ]:
# Qué se podría decir de las distribuciones de grado? Hay algunas más similares que otras?

### START CODE HERE
### END CODE HERE


## 2.2) Grado promedio de los vecinos de un vértice

In [ ]:
fig, ax = plt.subplots(4, 3, figsize=(20, 20))

for i, (k, g) in enumerate(graphs.items()):

  adn = [statistics.mean([g.degree(x) for x in g.neighbors(n)]) for n in range(g.vcount())]

  degree = g.degree()
  ax_ = ax[i // 3, i % 3]
  ax_.hist(adn)
  ax_.set_title(f"Graph {k} -- |V|= {g.vcount()}, |E| = {g.ecount()}")

## 2.3) Calcular la distribución de intermediación

In [ ]:
fig, ax = plt.subplots(4, 3, figsize=(20, 20))

for i, (k, g) in enumerate(graphs.items()):

  betweeness = g.betweenness()

  degree = g.degree()
  ax_ = ax[i // 3, i % 3]
  ax_.hist(betweeness)
  ax_.set_title(f"Graph {k} -- |V|= {g.vcount()}, |E| = {g.ecount()}")

In [ ]:
# Qué se podría decir de las distribuciones de intermediación? Hay algunas más similares que otras?

### START CODE HERE
### END CODE HERE


# 3) Calcular la cantidad de cliques de cada uno de los grafos

Una de las formas tradicionales de estudiar caracteristicas en distintos grafos es contar la cantidad de subgrafos de cada "tipo" que poseen.
Siendo los grafos completos uno de los grafos más simples de todos, es bastante usual calcular la cantidad de "cliques" que tiene un grafo.

In [ ]:
def count_unique_cliques(graph):
  """
  For each n >= 3, calculates the number of
  cliques of `graph` with size n.

  Returns
  -------
  count_cliques: dict
    Number of cliques for each n >=3.
  """
  count_cliques = defaultdict(int)
### START CODE HERE
### END CODE HERE
  return count_cliques

Ejemplo de uso de la función anterior para el grafo `a`.

El resultado esperado es `{3: 185, 4:27, 5:1}`.

In [ ]:
count_unique_cliques(graphs["a"]) 

Ahora guardamos en la lista `number_cliques` la cantidad de cliques para cada grafo (exceptuando los grafos grandes).

In [ ]:
number_cliques = {}
for k, g in graphs.items():
  start = time.perf_counter()

  if g.vcount() < 1000: # Dado que encontrar cliques es un problema NP-dificil, filtramos aquí para no usar los grafos con >=1000 vértices.
    number_cliques[k] = count_unique_cliques(g)

  elapsed = time.perf_counter() - start
  print(f"Elapsed time {k} with {g.vcount():d} nodes: {elapsed:.2f}")

In [ ]:
# presentamos los resultados en una tabla

df_cliques = pd.DataFrame(number_cliques).T.fillna(0).astype(int)
df_cliques["|V|"] = df_cliques.index.map(lambda x: graphs[x].vcount())
#df_cliques = df_cliques.sort_values("|V|", ascending=True)
display(df_cliques)

In [ ]:
# ¿Qué puedes decir de los grafos en base a la cantidad de cliques qué cada uno tiene?

### START CODE HERE
### END CODE HERE


# 4) Coeficiente de clustering

El coeficiente de clustering es una medida tradicional que describe la localidad de un vértice y a su vez puede distinguir muy bien entre distintos tipos de grafos.

In [ ]:
for k, g in graphs.items():
  cl = g.transitivity_undirected()
  print(f"Graph {k} with {g.vcount():>4} nodes has a clustering coefficient of: {cl:.4f}")

# 5) Diámetro y largo de camino promedio

In [ ]:
for k, g in graphs.items():
  print(f"Graph {k} with diameter {g.diameter():3>} average path length: {g.average_path_length()}")

# 6) "Golpe de vista"

En este útlimo experimento, vamos a pedirle que exporte todos los grafos con `1000` vértices a `Gephi` y que genere una visualización para cada grafo usando el mismo algoritmo (a su elección) de layout.

¿Le ayuda a diferenciarlos de la misma forma que las métricas?
¿Es mejor? ¿Es peor? ¿O es complementario?

# 7) Trabajo sobre la red del proyecto final

Habiendo realizado varias medidiciones sobre redes artificiales, es hora de trabajar sobre la red que será utilizada durante el proyecto (o sobre varias en caso de no haberse decidido por alguna aún).

Reproducir todas las métricas calculadas en este práctico (excepto la cantidad de compenentes conexas si sobrepasa un tiempo límite) y crear un informe detallando el resultado de los experimentos realizados en este práctico.

Este informe deberá subirlo al EVA, de acuerdo a las indicaciones que se encuentran en la sección del práctico.